In [1]:
from model.ctabgan import CTABGAN
from model.eval.evaluation import get_utility_metrics,stat_sim
import numpy as np
import pandas as pd
import glob

In [2]:
csv_path = "Real_Datasets/king.csv"
N = 2000
df = pd.read_csv(csv_path)

df_small = df.sample(n=N, random_state=42)
df_small.to_csv("king_subset.csv", index=False)
print(df.shape, "→", df_small.shape)

(21613, 20) → (2000, 20)


In [3]:
num_exp = 1
dataset = "king"
real_path = "king_subset.csv"
fake_file_root = "Fake_Datasets"


In [4]:
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20,
                 categorical_columns = ['bedrooms', "floors", 'waterfront', 'view', 'condition', 'grade','zipcode'],   
                 log_columns = [],
                 mixed_columns= {"sqft_basement":[0.0],  "yr_renovated":[0.0]},
                 general_columns= ["bathrooms", "sqft_living", "sqft_above", "yr_built", "long", "sqft_living15"],
                 non_categorical_columns= [],
                 integer_columns = [],
                 problem_type= {"Regression": "price"}) 

for i in range(num_exp):
    synthesizer.fit()

100%|██████████| 150/150 [1:42:30<00:00, 41.00s/it]

Finished training in 6153.62882399559  seconds.


In [8]:
for i in range(3):
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}_ctabplus.csv".format(exp=i), index= False)

In [9]:
fake_paths = glob.glob(fake_file_root+"/"+dataset+"/"+"*")

In [10]:
model_dict =  {"Regression":["l_reg","ridge","lasso","B_ridge"]}
result_mat = get_utility_metrics(real_path,fake_paths,"MinMax",model_dict, test_ratio = 0.20)

result_df  = pd.DataFrame(result_mat,columns=["Mean_Absolute_Percentage_Error","Explained_Varaince_Score","R2_Score"])
result_df.index = list(model_dict.values())[0]
result_df

,Mean_Absolute_Percentage_Error,Explained_Varaince_Score,R2_Score
l_reg,-0.071638,0.156723,0.158933
ridge,-0.069014,0.149059,0.149921
lasso,-0.071614,0.156661,0.158871
B_ridge,-0.419623,0.633084,0.822010


In [11]:
# ctabplus
insurance_categorical = ['bedrooms', "floors", 'waterfront', 'view', 'condition', 'grade','zipcode']
stat_res_avg = []
for fake_path in fake_paths:
    stat_res = stat_sim(real_path,fake_path,insurance_categorical)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg).mean(axis=0).reshape(1,3),columns=stat_columns)
stat_results

column:  date WD:  0.02361893017436429
column:  bedrooms JSD:  0.07977660074158183
column:  bathrooms WD:  0.022906583825747166
column:  sqft_living WD:  0.03152829519849304
column:  sqft_lot WD:  0.008873725839515094
column:  floors JSD:  0.06870524330147842
column:  waterfront JSD:  0.02333027103580538
column:  view JSD:  0.0502304715962923
column:  condition JSD:  0.04905579580854277
column:  grade JSD:  0.12919252538515635
column:  sqft_above WD:  0.043043842396111524
column:  sqft_basement WD:  0.01763793208148029
column:  yr_built WD:  0.014860620117187445
column:  yr_renovated WD:  0.03602876812577541
column:  zipcode JSD:  0.1327231917124526
column:  lat WD:  0.013734349886877504
column:  long WD:  0.02346992578846229
column:  sqft_living15 WD:  0.04195701203656619
column:  sqft_lot15 WD:  0.013093179813900287
column:  price WD:  0.024807853410526784
column:  date WD:  0.030631430939522725
column:  bedrooms JSD:  0.07909386053269281
column:  bathrooms WD:  0.022844643746103562


,Average WD (Continuous Columns,Average JSD (Categorical Columns),Correlation Distance
0,0.024275,0.078148,1.920398
